# Tweaking the cellpy file format

A `cellpy` file is a `hdf5`-type file.

From v.5 it contains five top-level directories.


```python

# from cellreader.py
raw_dir = prms._cellpyfile_raw
step_dir = prms._cellpyfile_step
summary_dir = prms._cellpyfile_summary
meta_dir = "/info"  # hard-coded
fid_dir = prms._cellpyfile_fid

# from prms.py
_cellpyfile_root = "CellpyData"
_cellpyfile_raw = "/raw"
_cellpyfile_step = "/steps"
_cellpyfile_summary = "/summary"
_cellpyfile_fid = "/fid"

```


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import cellpy

## Creating a fresh file from a raw-file

In [3]:
create_cellpyfile = True
filename_full = Path("/Users/jepe/cellpy_data/cellpyfiles/20181026_cen31_03_GITT_cc_01.h5")
filename_first = Path("/Users/jepe/cellpy_data/cellpyfiles/20181026_cen31_03_GITT_cc_01_a.h5")
rawfile_full = Path("/Users/jepe/cellpy_data/raw/20181026_cen31_03_GITT_cc_01.res")

In [5]:
if create_cellpyfile:
    c = cellpy.get(rawfile_full, mass=0.23)
    c.save(filename_full)
    c1, c2 = c.split(4)
    c1.save(filename_first)
else:
    c1 = cellpy.get(filename_first)
    c = cellpy.get(filename_full)
    

(cellpy) - Loading raw-file: /Users/jepe/cellpy_data/raw/20181026_cen31_03_GITT_cc_01.res
<class 'cellpy.readers.core.Cell'>
(cellpy) - Setting mass: 0.23
(cellpy) - Creating step table
(cellpy) - Creating summary data
(cellpy) - Assuming cycling in anode half-cell (discharge before charge) mode
(cellpy) - Using the following nominal capacity: 3579
(cellpy) - Created CellpyData object
(cellpy) -  -> /Users/jepe/cellpy_data/cellpyfiles/20181026_cen31_03_GITT_cc_01.h5
<class 'cellpy.readers.core.Cell'>
<class 'cellpy.readers.core.Cell'>


AttributeError: can't set attribute

In [ ]:
cellpy.log.setup_logging(default_level="DEBUG")

## Update

In [ ]:
c.cells

In [ ]:
c.cells[0].raw_data_files[0].last_data_point

In [ ]:
c3 = c.dev_update_from_raw(rawfile_full)

In [ ]:
c3_fids = c3.cell.raw_data_files

In [ ]:
[x.last_data_point for x in c3_fids]

In [ ]:
cc3 = c3.cells[0]

In [ ]:
c3.cells[0].raw_data_files[0].last_data_point

In [ ]:
id(c)

In [ ]:
id(c3)

In [ ]:
c3.get_cycle_numbers(0)

In [ ]:
c3.cells

In [ ]:
c3.cells[0]

In [ ]:
fids = c.cell.raw_data_files

In [ ]:
fids[0].last_data_point

In [ ]:
for f in fids: print(f)

## Looking at `cellpy`´s internal parameter

In [ ]:
from cellpy import prms

In [ ]:
parent_level = prms._cellpyfile_root
raw_dir = prms._cellpyfile_raw
step_dir = prms._cellpyfile_step
summary_dir = prms._cellpyfile_summary
meta_dir = "/info"  # hard-coded
fid_dir = prms._cellpyfile_fid

In [ ]:
raw_dir

In [ ]:
parent_level + raw_dir

## Looking at a `cellpy` file using `pandas`

In [ ]:
print(f'name: {filename.name}')
print(f"size: {filename.stat().st_size/1_048_576:0.2f} Mb")

In [ ]:
with pd.HDFStore(filename) as store:
    pprint(store.keys())

In [ ]:
store = pd.HDFStore(filename)

In [ ]:
m = store.select(parent_level + meta_dir)
s = store.select(parent_level + summary_dir)
t = store.select(parent_level + step_dir) 
f = store.select(parent_level + fid_dir)

In [ ]:
store.close()

In [ ]:
f.T

## Looking at a `cellpy` file using `cellpy`

In [ ]:
c = cellpy.get(filename)

In [ ]:
cc = c.cell

In [ ]:
fid = cc.raw_data_files[0]

In [ ]:
fid.last_data_point  # This should be used when I will implement reading only new data